# Initial setup

In [1]:
#importing packages needed for analysis
import os
import numpy as np
import pandas as pd
import math
from pandas import DataFrame
from itertools import product

path = os.getcwd()
parent = os.path.dirname(path)
print(path)
print(parent)
#os.mkdir(path+'\outputs')


C:\Users\CMARCY\OneDrive - Environmental Protection Agency (EPA)\Desktop\py_projects\plantfile\NEEDS_project
C:\Users\CMARCY\OneDrive - Environmental Protection Agency (EPA)\Desktop\py_projects\plantfile


# Reading in NEMS and NEEDS

In [15]:
#Reading in nems data (Note: this is only used for QA)
nems = pd.read_csv('background/ReEDS_generator_database_final_EIA-NEMS.csv', low_memory=False)
#print('number of columns =',nems.shape[1])
#print(nems.columns[0:26],nems.columns[26:54],nems.columns[115:])
#print()

pf = nems[['tech','pca','ct','resource_region','cap','Nuke60RetireYear','Nuke80RetireYear','NukeEarlyRetireYear','NukeRefRetireYear',
       'RetireYear','Commercial.Online.Year.Quarter','IsExistUnit','Fully.Loaded.Tested.Heat.Rate.Btu.kWh...Modeled',
       'Plant.NAICS.Description','UniqueID','Commercial.Online.Year','NukeRetireBin','EFDcd','UniqueID.1','RetireYearGiven']].copy()

pf['Commercial.Online.Year.Quarter'] = pf['Commercial.Online.Year.Quarter'].str[:4].astype(int)
#print('number of columns =',pf.shape[1])
#print(pf.head(2))
#print()


In [3]:
#Reading in needs data
needs_active = pd.read_csv('inputs/needs_v6_06-30-2020 - NEEDS v6_active.csv', low_memory=False)
needs_retire = pd.read_csv('inputs/needs_v6_06-30-2020 - NEEDS v6_Retired_Through2021.csv', low_memory=False)
needs_new = pd.read_csv('inputs/needs_v6_06-30-2020 - NEEDS v6_New_Capacity_Hardwired.csv', low_memory=False)

#checking columns
na = list(needs_active.columns)
nr = list(needs_retire.columns)
nn = list(needs_new.columns)
check = na==nr==nn
print('all columns match:',check)
print()

#counting rows
print('rows in active =',needs_active.shape[0])
print('rows in retire =',needs_retire.shape[0])
print('rows in new =',needs_new.shape[0])
rows_total = needs_active.shape[0]+needs_retire.shape[0]+needs_new.shape[0]
print('rows total =',rows_total)
print()

#appending the needs datasheets into a single dataframe
needs = pd.concat([needs_active,needs_retire,needs_new], ignore_index=True)
print('rows in needs =',needs.shape[0])
print('rows in needs = total rows:',needs.shape[0]==rows_total)
print()

#getting rid of spaces in the column name
needs.columns = needs.columns.str.replace(r'\s+', '_')

#dropping unused columns, comment out for final product
#needs = needs.drop(columns=['NOx_Comb_Control', 'NOx_Post-Comb_Control','SCR_Online_Year','SNCR_Online_Year','PM_Control',
#    'Mercury_Controls','ACI_Online_Year','Mercury_Controls_Efficiency','SO2_Permit_Rate_(lbs/mmBtu)','Mode_1_NOx_Rate_(lbs/mmBtu)',
#    'Mode_2_NOx_Rate_(lbs/mmBtu)','Mode_3_NOx_Rate_(lbs/mmBtu)','Mode_4_NOx_Rate_(lbs/mmBtu)','Hg_EMF_for_BIT','Hg_EMF_for_SUB',
#    'Hg_EMF_for_LIG','HCL_Removal','DSI_Unit','DSI_Online_Year','CCS','CCS_Removal','C2G','C2G_Online_Year','BART_Affected_Unit',
#    'Boiler/Generator/Committed_Unit','CAMD_Database_UnitID','Combustion_Turbine/IC_Engine','Region_Name','State_Code',
#    'County_Code','FIPS5','Firing','Bottom','Cogen?','Modeled_Fuels','FlueGasConditioning_Flag'])

print(needs.columns)
#needs.to_csv("outputs\\needs.csv")


all columns match: True

rows in active = 18191
rows in retire = 829
rows in new = 978
rows total = 19998

rows in needs = 19998
rows in needs = total rows: True

Index(['Plant_Name', 'UniqueID_Final', 'ORIS_Plant_Code',
       'Boiler/Generator/Committed_Unit', 'Unit_ID', 'CAMD_Database_UnitID',
       'PlantType', 'Combustion_Turbine/IC_Engine', 'Region_Name',
       'State_Name', 'State_Code', 'County', 'County_Code', 'FIPS5',
       'Capacity_(MW)', 'Heat_Rate_(Btu/kWh)', 'On_Line_Year',
       'Retirement_Year', 'Firing', 'Bottom', 'Cogen?', 'Modeled_Fuels',
       'Wet/DryScrubber', 'Scrubber_Online_Year', 'Scrubber_Efficiency',
       'NOx_Comb_Control', 'NOx_Post-Comb_Control', 'SCR_Online_Year',
       'SNCR_Online_Year', 'PM_Control', 'FlueGasConditioning_Flag',
       'Mercury_Controls', 'ACI_Online_Year', 'Mercury_Controls_Efficiency',
       'SO2_Permit_Rate_(lbs/mmBtu)', 'Mode_1_NOx_Rate_(lbs/mmBtu)',
       'Mode_2_NOx_Rate_(lbs/mmBtu)', 'Mode_3_NOx_Rate_(lbs/mmBtu)',
  

# Matching NEEDS data to ReEDS lookups

In [4]:
#Adding ReEDS Regional Data

#this table matches ReEDS regions to counties
pca_lookup = pd.read_csv('inputs/pca_lookup.csv')

#modifying NEEDS to be able to match with ReEDS
needs['County_lower'] = needs['County'].str.lower()
needs['County_lower'] = needs['County_lower'].str.replace(' ','')

#modifying ReEDS to be able to match with NEEDS
pca_lookup = pca_lookup.rename(columns={'State':'State_Name'})
pca_lookup['County_lower'] = pca_lookup['County'].str.replace('St. ','St ') \
    .str.replace(' ','').str.lower()
pca_lookup = pca_lookup.drop(columns=['County'])
#print(pca_lookup.head())

#merging needs and the pca lookup table
needs_pca = pd.merge(needs,pca_lookup,on=['County_lower','State_Name'],how='left')
#this column was only created to facilitate the merge and is no longer needed
needs_pca = needs_pca.drop(columns=['County_lower']) 
#print(needs_pca.head(2))

#identify mismatched county data, this DF should be empty
pca_na = needs_pca[needs_pca['pca'].isnull()]
pca_na_id = pca_na[['County','State_Name']]
print('Number of rows with mismatched counties:',pca_na_id.shape[0])
pca_na_id = pca_na_id.groupby(['County','State_Name']).first().reset_index()
print('Number of mismatched counties:',pca_na_id.shape[0])
print()
print(pca_na_id)


Number of rows with mismatched counties: 0
Number of mismatched counties: 0

Empty DataFrame
Columns: [index]
Index: []


In [5]:
#Removing rows that come online and retire in the same year

needs_pca['On_Line_Year'] = needs_pca['On_Line_Year'].astype(int)
needs_pca['Online_Yr_OG'] = needs_pca['On_Line_Year']
needs_pca['Retire_Yr_OG'] = needs_pca['Retirement_Year']

needs_pca['Retirement_Year'].fillna(9999, inplace = True)
needs_pca['Retirement_Year'] = needs_pca['Retirement_Year'].astype(int)
needs_pca['Retire_Age_OG'] = needs_pca['Retirement_Year'] - needs_pca['On_Line_Year']
has_bad_years = (needs_pca['Retire_Age_OG'] <= 0)
bad_years = needs_pca[has_bad_years].copy()
print('number of rows removed with zero lifetimes:',len(bad_years))
#print(bad_years.head())
#bad_years.to_csv("outputs\\bad_years.csv")

has_good_years = (needs_pca['Retire_Age_OG'] > 0)
needs_years = needs_pca[has_good_years].copy()
print('before:',len(needs_pca))
print('after:',len(needs_years))
needs_years = needs_years.drop(columns=['Retire_Age_OG'])
#needs_years.to_csv("outputs\\good_years.csv")


number of rows removed with zero lifetimes: 34
before: 20004
after: 19970


# Adding ReEDS Tech Data

In [6]:
#export ReEDS tech names in order to make the lookup table to be imported
unique_pt = pd.Series(needs_years['PlantType'].unique())
#print(unique_pt)
#unique_pt.to_csv("inputs\\unique_pt.csv")

tech_lookup = pd.read_csv('inputs/tech_lookup.csv')
#print(tech_lookup)
#print()

tech_lookup.columns = ['PlantType', 'tech'] 
needs_tech = pd.merge(needs_years,tech_lookup,on=['PlantType'],how='left')
#print(needs_tech.head(2))

#Tech: remove others
is_other = (needs_tech['tech'] == 'other')
other = needs_tech[is_other].copy()
#other.to_csv('outputs\\other.csv')
print('number of rows removed where tech is other:', len(other))
print('before:',needs_tech.shape[0])
needs_tech = needs_tech[~is_other]
print('after:',needs_tech.shape[0])
print()

#Tech: creates DUPV
#print('rows with PV technology:',needs_tech[needs_tech['tech'] == 'UPV'].shape[0])
upv = (needs_tech['tech'] == 'UPV') & (needs_tech['Capacity_(MW)'] >= 5)
dupv = (needs_tech['tech'] == 'UPV') & (needs_tech['Capacity_(MW)'] < 5)
needs_tech.loc[upv,'tech'] = 'UPV'
needs_tech.loc[dupv,'tech'] = 'DUPV'
#print('rows with UPV technology:',needs_tech[upv].shape[0])
#print('rows with DUPV technology:',needs_tech[dupv].shape[0])

#Tech: defines CoalOldScr, CoalOldUns, and coal-new
#default coal technology is CoalOldUns, using this code to differentiate between coal labels

#creating separate DF of just scrubbed coal units, setting their online year to when the scrubber came online
is_scrb = (needs_tech['tech'] == 'CoalOldUns') & (needs_tech['Scrubber_Online_Year'].isnull() == False)
scrb = needs_tech[is_scrb].copy()
scrb['tech'] = 'CoalOldScr'
scrb['Scrub_Match'] = 'Scrb'
scrb['On_Line_Year'] = scrb['Scrubber_Online_Year'].astype(int)
scrb.loc[(scrb['Wet/DryScrubber'] == 'Wet Scrubber') & (scrb['Scrubber_Online_Year'] >= 1995),'tech'] = 'coal-new' 
#print(scrb.head())

#retiring coal plants in main file at year scrubber comes online
needs_tech['Scrub_Match'] = ''
needs_tech.loc[is_scrb,'Scrub_Match'] = 'Unscrb'
needs_tech.loc[is_scrb,'Retirement_Year'] = needs_tech['Scrubber_Online_Year']
needs_tech.loc[is_scrb,'RetireYearGiven'] = True
needs_tech.loc[is_scrb,'Wet/DryScrubber'] = np.nan
needs_tech.loc[is_scrb,'Scrubber_Online_Year'] = np.nan
needs_tech.loc[is_scrb,'Scrubber_Efficiency'] = np.nan
#print(needs_tech[is_scrb].head())

#removing techs that first came on as scrubbed and were never unscrubbed
never_unsb = (needs_tech['On_Line_Year'] >= needs_tech['Retirement_Year'])
unsb_err = needs_tech[never_unsb].copy()
#unsb_err.to_csv('outputs\\unsb_err.csv')
print('number of rows removed with zero lifetimes:', len(unsb_err))
print('before:',needs_tech.shape[0])
needs_tech = needs_tech[~never_unsb]
print('after:',needs_tech.shape[0])
print()

#appending the scrb dataframe, creating a new entry for coal plants at year scrubber comes online
print('number of scrubbed rows added:', scrb.shape[0])
print('before:',needs_tech.shape[0])
needs_tech = needs_tech.append(scrb)
needs_tech.reset_index(drop=True,inplace=True)
print('after:',needs_tech.shape[0])
print()

#Note: This code is used to identify mismatched tech data, this DF should be empty
tech_na = needs_tech[needs_tech['tech'].isnull()]
print('Number of rows with mismatched tech:',tech_na.shape[0])


number of rows removed where tech is other: 323
before: 19970
after: 19647

number of rows removed with zero lifetimes: 192
before: 19647
after: 19455

number of scrubbed rows added: 503
before: 19455
after: 19958

Number of rows with mismatched tech: 0


# Adding ReEDS Retirement Data

In [7]:
#create columns for RetireYearGiven and Retire_Age_OG
needs_retire = needs_tech.copy()
needs_retire['Retirement_Year'].fillna(9999, inplace = True)
no_retires = needs_retire['Retirement_Year'] == 9999
needs_retire.loc[no_retires,'RetireYearGiven'] = False
needs_retire.loc[~no_retires,'RetireYearGiven'] = True

needs_retire['Retire_Age'] = needs_retire['Retirement_Year'] - needs_retire['On_Line_Year']
needs_retire.loc[no_retires,'Retire_Age'] = 9999

#non-nuclear retirements

lifetimes = pd.read_csv('inputs/lifetimes.csv')
lifetimes.set_index('tech',inplace=True)
#print(lifetimes)
#needs_retire.to_csv('outputs\\retire.csv')

print('length of loop:',len(needs_retire))
print()

for i in range(0,len(needs_retire),1):
    #print(i)
    if needs_retire.loc[i,'Retirement_Year'] == 9999:
        tech = needs_retire.loc[i,'tech']
        size = needs_retire.loc[i,'Capacity_(MW)']
        if size >= 100:
            lifetime = lifetimes.loc[tech,'lifetime_big']
        elif size < 100:
            lifetime = lifetimes.loc[tech,'lifetime_small']
        needs_retire.loc[i,'Retirement_Year'] = needs_retire.loc[i,'Online_Yr_OG'] + lifetime
    elif needs_retire.loc[i,'Retirement_Year'] != 9999:
        pass

#remove wind retirements before 2010
is_old = (needs_retire['tech'] == 'wind-ons') & (needs_retire['Retirement_Year'] <= 2010)
old_wind = needs_retire[is_old].copy()
#is_old.to_csv('outputs\\is_old.csv')
print('number of rows removed where wind is already retired:', len(old_wind))
print('before:',needs_retire.shape[0])
needs_retire = needs_retire[~is_old]
print('after:',needs_retire.shape[0])
print()

needs_retire['Retirement_Year'] = needs_retire['Retirement_Year'].astype(int)
print(needs_retire.columns)


length of loop: 19958

number of rows removed where wind is already retired: 1
before: 19958
after: 19957

Index(['Plant_Name', 'UniqueID_Final', 'ORIS_Plant_Code',
       'Boiler/Generator/Committed_Unit', 'Unit_ID', 'CAMD_Database_UnitID',
       'PlantType', 'Combustion_Turbine/IC_Engine', 'Region_Name',
       'State_Name', 'State_Code', 'County', 'County_Code', 'FIPS5',
       'Capacity_(MW)', 'Heat_Rate_(Btu/kWh)', 'On_Line_Year',
       'Retirement_Year', 'Firing', 'Bottom', 'Cogen?', 'Modeled_Fuels',
       'Wet/DryScrubber', 'Scrubber_Online_Year', 'Scrubber_Efficiency',
       'NOx_Comb_Control', 'NOx_Post-Comb_Control', 'SCR_Online_Year',
       'SNCR_Online_Year', 'PM_Control', 'FlueGasConditioning_Flag',
       'Mercury_Controls', 'ACI_Online_Year', 'Mercury_Controls_Efficiency',
       'SO2_Permit_Rate_(lbs/mmBtu)', 'Mode_1_NOx_Rate_(lbs/mmBtu)',
       'Mode_2_NOx_Rate_(lbs/mmBtu)', 'Mode_3_NOx_Rate_(lbs/mmBtu)',
       'Mode_4_NOx_Rate_(lbs/mmBtu)', 'Hg_EMF_for_BIT', 'H

In [8]:
#nuclear retirements

#exported NEEDS nuke names in order to adjust the lookup table that is imported next
unique_nuke = needs_retire[needs_retire['tech'] == 'nuclear']
unique_nuke2 = pd.Series(unique_nuke['Plant_Name'].unique())
print('Number of nuke plants in NEEDS:',len(unique_nuke2))
#unique_nuke2.to_csv("inputs\\unique_nuke.csv")

nuke_mapper = pd.read_csv('inputs/nuke_mapper.csv')
#print(nuke_mapper.head())
print('Number of nuke plants in mapper:',len(nuke_mapper))

needs_nuke = pd.merge(needs_retire,nuke_mapper,on=['Plant_Name','tech'],how='left')
nukebin = needs_nuke[needs_nuke['NukeRetireBin'].notnull()]
#print(nukebin.head())
print('Number of used nuke plants from mapper:',len(nukebin['Plant_Name'].unique()))

#NEEDS defaults to 80-yr lifespan for nuclear plants not reporting a retirement year
needs_nuke['Nuke80RetireYear'] = needs_nuke['Retirement_Year'].astype(int)

#ReEDS defaults to 60-yr lifespan for nuclear plants not reporting a retirement year, adjusted here
nuke_80 = (needs_nuke['tech'] == 'nuclear') & (needs_nuke['Retire_Age'] == 80)
needs_nuke.loc[nuke_80,'Retirement_Year'] -= 20
needs_nuke['Nuke60RetireYear'] = needs_nuke['Retirement_Year'].astype(int)

#nuclear plants in needs with 80 year lifespan are actually plants without reported retirement dates
#needs_nuke.loc[nuke_80,'RetireYearGiven'] = False

#ReEDS NukeRefRetireYear and NukeEarlyRetireYear change retirement assumptions based on nuke bins
nukes1 = nuke_80 & (needs_nuke['NukeRetireBin'] == 1)
nukes2 = nuke_80 & (needs_nuke['NukeRetireBin'] == 2)
#print(nukes1.sum())
#print(nukes2.sum())

needs_nuke['NukeEarlyRetireYear'] = needs_nuke['Retirement_Year'].astype(int)
needs_nuke['NukeRefRetireYear'] = needs_nuke['Retirement_Year'].astype(int)
#nuke_check = needs_nuke[nuke_80].copy()
#print(nuke_check['NukeEarlyRetireYear'].describe())
needs_nuke.loc[nukes1,'NukeEarlyRetireYear'] -= 10
needs_nuke.loc[nukes2,'NukeRefRetireYear'] += 20

#nuke_check = needs_nuke[nuke_80].copy()
#print(nuke_check['NukeEarlyRetireYear'].describe())


Number of nuke plants in NEEDS: 58
Number of nuke plants in mapper: 66
Number of used nuke plants from mapper: 58


In [9]:
#Final formatting 
needs_final = needs_nuke.copy()

IsExistUnit = (needs_final['On_Line_Year'] <= 2010)
needs_final.loc[IsExistUnit,'IsExistUnit'] = True
needs_final.loc[~IsExistUnit,'IsExistUnit'] = False

needs_final = needs_final.rename(columns={'On_Line_Year':'Commercial.Online.Year',
                                          'Capacity_(MW)':'cap',
                                          'Retirement_Year':'RetireYear',
                                          'Heat_Rate_(Btu/kWh)':'Fully.Loaded.Tested.Heat.Rate.Btu.kWh...Modeled',
                                          'UniqueID_Final':'UniqueID'})

#creating dummy variables for cooling technologies
needs_final['ct'] = 'none'
needs_final['ctt'] = 'none'
needs_final['wst'] = 'none'
needs_final['coolingwatertech'] = 'none'
needs_final['Plant.NAICS.Description'] = 'Utilities'
needs_final['Commercial.Online.Year.Quarter'] = needs_final['Commercial.Online.Year'].astype(str) + '-1'
print(needs_final.columns)
#print(needs_final.dtypes)
needs_final.to_csv('outputs\\needs_final.csv')


Index(['Plant_Name', 'UniqueID', 'ORIS_Plant_Code',
       'Boiler/Generator/Committed_Unit', 'Unit_ID', 'CAMD_Database_UnitID',
       'PlantType', 'Combustion_Turbine/IC_Engine', 'Region_Name',
       'State_Name', 'State_Code', 'County', 'County_Code', 'FIPS5', 'cap',
       'Fully.Loaded.Tested.Heat.Rate.Btu.kWh...Modeled',
       'Commercial.Online.Year', 'RetireYear', 'Firing', 'Bottom', 'Cogen?',
       'Modeled_Fuels', 'Wet/DryScrubber', 'Scrubber_Online_Year',
       'Scrubber_Efficiency', 'NOx_Comb_Control', 'NOx_Post-Comb_Control',
       'SCR_Online_Year', 'SNCR_Online_Year', 'PM_Control',
       'FlueGasConditioning_Flag', 'Mercury_Controls', 'ACI_Online_Year',
       'Mercury_Controls_Efficiency', 'SO2_Permit_Rate_(lbs/mmBtu)',
       'Mode_1_NOx_Rate_(lbs/mmBtu)', 'Mode_2_NOx_Rate_(lbs/mmBtu)',
       'Mode_3_NOx_Rate_(lbs/mmBtu)', 'Mode_4_NOx_Rate_(lbs/mmBtu)',
       'Hg_EMF_for_BIT', 'Hg_EMF_for_SUB', 'Hg_EMF_for_LIG', 'HCL_Removal',
       'DSI_Unit', 'DSI_Online_Yea

# QA Review

In [10]:
#high level QA

needs_on = needs_final.groupby(['tech','Commercial.Online.Year'],as_index=False)['cap'].sum()
minyr = int(needs_on['Commercial.Online.Year'].min())
unique_tech = pd.Series(needs_final['tech'].unique())

needs_start = pd.DataFrame(unique_tech, columns=['tech'])
needs_start['Commercial.Online.Year'] = minyr - 1
needs_start['cap'] = 0.0

needs_on = pd.merge(needs_on,needs_start,on=['tech','Commercial.Online.Year','cap'],how='outer')
needs_on = needs_on.rename(columns={'Commercial.Online.Year':'year','cap':'cap_online'})
print(needs_on.head())
print()

needs_off = needs_final.groupby(['tech','RetireYear'],as_index=False)['cap'].sum()
needs_off['RetireYear'] = needs_off['RetireYear'].astype(int) 
needs_off = needs_off.rename(columns={'RetireYear':'year','cap':'cap_retire'})
maxyr = int(needs_off['year'].max())
print(needs_off.tail())
print()

years = pd.Series(range(minyr-1,maxyr))
needs_allyrs = pd.DataFrame(product(unique_tech,years), columns=['tech','year'])
needs_allyrs['cap'] = 0.0 

needs_allyrs = pd.merge(needs_allyrs,needs_on,on=['tech','year'],how='left')
needs_allyrs = pd.merge(needs_allyrs,needs_off,on=['tech','year'],how='left')
needs_allyrs = needs_allyrs.fillna(0)
needs_allyrs['diff'] = needs_allyrs['cap_online'] - needs_allyrs['cap_retire']

for i in range(0,len(needs_allyrs),1):
    if needs_allyrs.loc[i,'year'] != minyr-1:
        needs_allyrs.loc[i,'cap'] = needs_allyrs.loc[i-1,'cap'] + needs_allyrs.loc[i,'diff']
    elif needs_allyrs.loc[i,'year'] == minyr-1:
        needs_allyrs.loc[i,'cap'] = 0.0

check = needs_allyrs[needs_allyrs['cap'] != 0.0]
print(check.tail())

#check.to_csv('outputs\\needs_check.csv')
needs_check = check.copy()
needs_check['model'] = 'needs'

#import seaborn as sns
#sns.scatterplot('year', 'cap', data=needs_allyrs, hue='tech')


         tech  year  cap_online
0  CoalOldScr  1963        75.0
1  CoalOldScr  1973       116.0
2  CoalOldScr  1974       670.0
3  CoalOldScr  1975       360.0
4  CoalOldScr  1976      2499.0

         tech  year  cap_retire
848  wind-ons  2047      6929.9
849  wind-ons  2048      7023.4
850  wind-ons  2049      6337.2
851  wind-ons  2050      1392.4
852  wind-ons  2054       500.0

            tech  year           cap  cap_online  cap_retire  diff
4365  CoalOldScr  2115  5.456968e-12         0.0         0.0   0.0
4366  CoalOldScr  2116  5.456968e-12         0.0         0.0   0.0
4367  CoalOldScr  2117  5.456968e-12         0.0         0.0   0.0
4368  CoalOldScr  2118  5.456968e-12         0.0         0.0   0.0
4369  CoalOldScr  2119  5.456968e-12         0.0         0.0   0.0


In [11]:
#print('NEMS ORIGINAL')
#print('number of columns =',nems.shape[1])
#print(nems.columns[0:26],nems.columns[26:54],nems.columns[115:])
#print('note: incomplete list of columns printed here')
#print()
print('NEMS COLUMNS SELECTED')
print('number of columns =',pf.shape[1])
print(pf.columns)
#print(pf.head(2))
#print()


NEMS COLUMNS SELECTED
number of columns = 20
Index(['tech', 'pca', 'ct', 'resource_region', 'cap', 'Nuke60RetireYear',
       'Nuke80RetireYear', 'NukeEarlyRetireYear', 'NukeRefRetireYear',
       'RetireYear', 'Commercial.Online.Year.Quarter', 'IsExistUnit',
       'Fully.Loaded.Tested.Heat.Rate.Btu.kWh...Modeled',
       'Plant.NAICS.Description', 'UniqueID', 'Commercial.Online.Year',
       'NukeRetireBin', 'EFDcd', 'UniqueID.1', 'RetireYearGiven'],
      dtype='object')


In [12]:
#high level QA

nems_on = pf.groupby(['tech','Commercial.Online.Year.Quarter'],as_index=False)['cap'].sum()
minyr = int(nems_on['Commercial.Online.Year.Quarter'].min())
unique_tech = pd.Series(pf['tech'].unique())

nems_start = pd.DataFrame(unique_tech, columns=['tech'])
nems_start['Commercial.Online.Year.Quarter'] = minyr - 1
nems_start['cap'] = 0.0

nems_on = pd.merge(nems_on,nems_start,on=['tech','Commercial.Online.Year.Quarter','cap'],how='outer')
nems_on = nems_on.rename(columns={'Commercial.Online.Year.Quarter':'year','cap':'cap_online'})
print(nems_on.head())
print()

nems_off = pf.groupby(['tech','RetireYear'],as_index=False)['cap'].sum()
nems_off['RetireYear'] = nems_off['RetireYear'].astype(int) 
nems_off = nems_off.rename(columns={'RetireYear':'year','cap':'cap_retire'})
maxyr = int(nems_off['year'].max())
print(nems_off.tail())
print()

years = pd.Series(range(minyr-1,maxyr))
nems_allyrs = pd.DataFrame(product(unique_tech,years), columns=['tech','year'])
nems_allyrs['cap'] = 0.0 

nems_allyrs = pd.merge(nems_allyrs,nems_on,on=['tech','year'],how='left')
nems_allyrs = pd.merge(nems_allyrs,nems_off,on=['tech','year'],how='left')
nems_allyrs = nems_allyrs.fillna(0)
nems_allyrs['diff'] = nems_allyrs['cap_online'] - nems_allyrs['cap_retire']

for i in range(0,len(nems_allyrs),1):
    if nems_allyrs.loc[i,'year'] != minyr-1:
        nems_allyrs.loc[i,'cap'] = nems_allyrs.loc[i-1,'cap'] + nems_allyrs.loc[i,'diff']
    elif nems_allyrs.loc[i,'year'] == minyr-1:
        nems_allyrs.loc[i,'cap'] = 0.0

check = nems_allyrs[nems_allyrs['cap'] != 0.0]
#print(check.head(50))

#check.to_csv('outputs\\nems_check.csv')
nems_check = check.copy()
nems_check['model'] = 'nems'

check_final = nems_check.append(needs_check)
check_final.to_csv('outputs\\check.csv')

#import seaborn as sns
#sns.scatterplot('year', 'cap', data=nems_allyrs, hue='tech')


         tech  year  cap_online
0  CoalOldScr  1950       250.8
1  CoalOldScr  1951        35.0
2  CoalOldScr  1952       190.5
3  CoalOldScr  1953       361.0
4  CoalOldScr  1954      1249.0

         tech  year  cap_retire
896  wind-ons  2047      6311.9
897  wind-ons  2048      7600.2
898  wind-ons  2049     11275.6
899  wind-ons  2050      4819.1
900  wind-ons  2051       180.0



In [13]:
print(needs_final.columns)
needs_review = needs_final[['ORIS_Plant_Code','Unit_ID','pca','tech','cap','Wet/DryScrubber','Scrubber_Efficiency',
                            'Online_Yr_OG','Scrubber_Online_Year','RetireYear']].copy()
#nems_review = nems[['UniqueID','pca','tech','cap','Commercial.Online.Year','RetireYear']].copy()

Index(['Plant_Name', 'UniqueID', 'ORIS_Plant_Code',
       'Boiler/Generator/Committed_Unit', 'Unit_ID', 'CAMD_Database_UnitID',
       'PlantType', 'Combustion_Turbine/IC_Engine', 'Region_Name',
       'State_Name', 'State_Code', 'County', 'County_Code', 'FIPS5', 'cap',
       'Fully.Loaded.Tested.Heat.Rate.Btu.kWh...Modeled',
       'Commercial.Online.Year', 'RetireYear', 'Firing', 'Bottom', 'Cogen?',
       'Modeled_Fuels', 'Wet/DryScrubber', 'Scrubber_Online_Year',
       'Scrubber_Efficiency', 'NOx_Comb_Control', 'NOx_Post-Comb_Control',
       'SCR_Online_Year', 'SNCR_Online_Year', 'PM_Control',
       'FlueGasConditioning_Flag', 'Mercury_Controls', 'ACI_Online_Year',
       'Mercury_Controls_Efficiency', 'SO2_Permit_Rate_(lbs/mmBtu)',
       'Mode_1_NOx_Rate_(lbs/mmBtu)', 'Mode_2_NOx_Rate_(lbs/mmBtu)',
       'Mode_3_NOx_Rate_(lbs/mmBtu)', 'Mode_4_NOx_Rate_(lbs/mmBtu)',
       'Hg_EMF_for_BIT', 'Hg_EMF_for_SUB', 'Hg_EMF_for_LIG', 'HCL_Removal',
       'DSI_Unit', 'DSI_Online_Yea

## Merging data

In [14]:
#Merge code is not finalized 

needs['needs']=1
nems['nems']=1
#print(needs.head())
needs = needs.rename(columns={'ORIS_Plant_Code':'PID','Unit_ID':'UID'})
#needs = needs[['PID','UID','PlantType','Capacity_(MW)','needs']]
#print(needs.columns)
print(needs.shape[0])

nems = nems.rename(columns={'T_PID':'PID',' T_UID':'UID','tech':'tech_nems'})
#nems = nems[['PID','UID','tech_nems','cap','nems']]
#print(nems.columns)
print(nems.shape[0])

combo = pd.merge(needs,nems,on=['PID','UID'],how='outer')
print(combo.shape[0])
#combo.to_csv('outputs\\combo.csv')

19998
23039
33174
